In [2]:
# clear workspace
rm(list=ls())

# load libraries
library(readr)
library(dplyr)
library(magrittr)
library(ggplot2)
library(lfe)
library(stargazer)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: Matrix

Please cite as: 

 Hlavac, Marek (2015). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2. http://CRAN.R-project.org/package=stargazer 



In [3]:
# set absolute directory to be source file directory
# working_dir <-
#   rstudioapi::getActiveDocumentContext()$path %>%
#   dirname()
# setwd(working_dir)
working_dir <- getwd()

# define paths
IN_DIR <- "data/GiveDirectly/merged.csv"
OUT_DIR <- "data/GiveDirectly/Output/"

# load data
df <- read_csv(IN_DIR)

Parsed with column specification:
cols(
  .default = col_double(),
  s1_hhid_key = col_character(),
  s6_q1a_electype = col_character()
)
See spec(...) for full column specifications.


In [4]:
names(df)

[1] "s1_hhid_key"               "s6_q1_haselectricity"     
 [3] "s6_q1a_electype"           "s6_q6_acresowned"         
 [5] "s6_q6b_agacresowned"       "s6_q7_rentedland"         
 [7] "s6_q7a_acresrented"        "s6_q7b_monthsrented"      
 [9] "s6_q7ci_landrent"          "s6_q8_landrenting"        
[11] "s6_q8a_acresrenting"       "s6_q8b_agacresrenting"    
[13] "s6_q8c_monthsrenting"      "s19_gps_latitude"         
[15] "s19_gps_longitude"         "treat"                    
[17] "hi_sat"                    "p1_assets_wins"           
[19] "p1_6_nonthatchedroof"      "net_asset_value_home_wins"
[21] "p2_consumption_wins"       "p3_1_agprofit_wins"       
[23] "land_agacresowned_BL"      "land_agvalue_BL"          
[25] "aglandrentalcost_BL"       "aglanduse_BL"             
[27] "p1_assets_wins_BL"         "net_asset_value_home_BL"  
[29] "p1_6_nonthatchedroof_BL"   "satcluster"               
[31] "all_sat_house"             "colored_sat_house"        
[33] "metal_sat_house"           "thatched_sat_house"       
[35] "all_sat_size_mean"         "colored_sat_size_mean"    
[37] "metal_sat_size_mean"       "thatched_sat_size_mean"   
[39] "all_sat_size_sum"          "colored_sat_size_sum"     
[41] "metal_sat_size_sum"        "thatched_sat_size_sum"    
[43] "nightlight"

In [6]:
reg1 <- felm(nightlight ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg2 <- felm(nightlight ~ treat + hi_sat | 0 | 0 | satcluster, df[df$p1_6_nonthatchedroof_BL == 0,])

output <- stargazer(reg1, reg2)
cat(paste0(output, "\n"), file=paste0(OUT_DIR, "nl.tex"))


% Table created by stargazer v.5.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Thu, Dec 05, 2019 - 02:28:04 AM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{2}{c}{\textit{Dependent variable:}} \\ 
\cline{2-3} 
\\[-1.8ex] & \multicolumn{2}{c}{nightlight} \\ 
\\[-1.8ex] & (1) & (2)\\ 
\hline \\[-1.8ex] 
 treat & 0.028 & 0.011 \\ 
  & (0.077) & (0.070) \\ 
  & & \\ 
 hi\_sat & 0.095 & 0.014 \\ 
  & (0.083) & (0.070) \\ 
  & & \\ 
 Constant & 0.480$^{***}$ & 0.495$^{***}$ \\ 
  & (0.046) & (0.048) \\ 
  & & \\ 
\hline \\[-1.8ex] 
Observations & 5,233 & 4,529 \\ 
R$^{2}$ & 0.001 & 0.00002 \\ 
Adjusted R$^{2}$ & 0.0001 & $-$0.0004 \\ 
Residual Std. Error & 2.253 (df = 5230) & 2.075 (df = 4526) \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{2}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 


In [7]:
reg1 <- felm(all_sat_house ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg2 <- felm(all_sat_house ~ treat + hi_sat | 0 | 0 | satcluster, df[df$p1_6_nonthatchedroof_BL == 0,])
reg3 <- felm(all_sat_size_mean ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg4 <- felm(all_sat_size_mean ~ treat + hi_sat | 0 | 0 | satcluster, df[df$p1_6_nonthatchedroof_BL == 0,])
reg5 <- felm(all_sat_size_sum ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg6 <- felm(all_sat_size_sum ~ treat + hi_sat | 0 | 0 | satcluster, df[df$p1_6_nonthatchedroof_BL == 0,])

output <- stargazer(reg1, reg2, reg3, reg4, reg5, reg6,
                    dep.var.labels=c("No. of Houses",
                                     "Mean Size of Houses",
                                     "Total Area of Houses"),
                    column.labels=c("Full", "Eligible",
                                    "Full", "Eligible",
                                    "Full", "Eligible"),
                    covariate.labels=c("Treatment Village", "High Saturation Sublocation", "Constant"))
cat(paste0(output, "\n"), file=paste0(OUT_DIR, "all_te.tex"))


% Table created by stargazer v.5.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Thu, Dec 05, 2019 - 02:28:34 AM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{6}{c}{\textit{Dependent variable:}} \\ 
\cline{2-7} 
\\[-1.8ex] & \multicolumn{2}{c}{No. of Houses} & \multicolumn{2}{c}{Mean Size of Houses} & \multicolumn{2}{c}{Total Area of Houses} \\ 
 & Full & Eligible & Full & Eligible & Full & Eligible \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6)\\ 
\hline \\[-1.8ex] 
 Treatment Village & 0.330$^{***}$ & 0.342$^{***}$ & 5.154$^{*}$ & 6.498$^{**}$ & 97.637$^{***}$ & 104.180$^{***}$ \\ 
  & (0.103) & (0.106) & (3.103) & (2.969) & (34.551) & (32.421) \\ 
  & & & & & & \\ 
 High Saturation Sublocation & 0.141 & 0.120 & 1.498 & 1.610 & 1.326 & $-$3.843 \\ 
  & (0.188) & (0.193) & (3.772) & (3.668) & (41.850) & (38.928) \\ 
  & & & & & 

In [8]:
reg1 <- felm(metal_sat_house ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg2 <- felm(metal_sat_house ~ treat + hi_sat | 0 | 0 | satcluster, df[df$p1_6_nonthatchedroof_BL == 0,])
reg3 <- felm(metal_sat_size_mean ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg4 <- felm(metal_sat_size_mean ~ treat + hi_sat | 0 | 0 | satcluster, df[df$p1_6_nonthatchedroof_BL == 0,])
reg5 <- felm(metal_sat_size_sum ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg6 <- felm(metal_sat_size_sum ~ treat + hi_sat | 0 | 0 | satcluster, df[df$p1_6_nonthatchedroof_BL == 0,])

output <- stargazer(reg1, reg2, reg3, reg4, reg5, reg6,
                    dep.var.labels=c("No. of Houses w/ Metal Roof",
                                     "Mean Size of Houses w/ Metal Roof",
                                     "Area Covered by Metal Roof"),
                    column.labels=c("Full", "Eligible",
                                    "Full", "Eligible",
                                    "Full", "Eligible"),
                    covariate.labels=c("Treatment Village", "High Saturation Sublocation", "Constant"))
cat(paste0(output, "\n"), file=paste0(OUT_DIR, "metal_te.tex"))


% Table created by stargazer v.5.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Thu, Dec 05, 2019 - 02:28:34 AM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{6}{c}{\textit{Dependent variable:}} \\ 
\cline{2-7} 
\\[-1.8ex] & \multicolumn{2}{c}{No. of Houses w/ Metal Roof} & \multicolumn{2}{c}{Mean Size of Houses w/ Metal Roof} & \multicolumn{2}{c}{Area Covered by Metal Roof} \\ 
 & Full & Eligible & Full & Eligible & Full & Eligible \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6)\\ 
\hline \\[-1.8ex] 
 Treatment Village & 0.587$^{***}$ & 0.602$^{***}$ & 3.153 & 3.854 & 133.397$^{***}$ & 137.282$^{***}$ \\ 
  & (0.097) & (0.098) & (3.578) & (3.707) & (33.678) & (31.093) \\ 
  & & & & & & \\ 
 High Saturation Sublocation & 0.240 & 0.219 & $-$4.306 & $-$2.661 & 3.295 & 1.210 \\ 
  & (0.161) & (0.166) & (5.823) & (5.213) & (35.639) & 

In [9]:
reg1 <- felm(thatched_sat_house ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg2 <- felm(thatched_sat_house ~ treat + hi_sat | 0 | 0 | satcluster, df[df$p1_6_nonthatchedroof_BL == 0,])
reg3 <- felm(thatched_sat_size_mean ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg4 <- felm(thatched_sat_size_mean ~ treat + hi_sat | 0 | 0 | satcluster, df[df$p1_6_nonthatchedroof_BL == 0,])
reg5 <- felm(thatched_sat_size_sum ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg6 <- felm(thatched_sat_size_sum ~ treat + hi_sat | 0 | 0 | satcluster, df[df$p1_6_nonthatchedroof_BL == 0,])

output <- stargazer(reg1, reg2, reg3, reg4, reg5, reg6,
                    dep.var.labels=c("No. of Houses w/ Thatched Roof",
                                     "Mean Size of Houses w/ Thatched Roof",
                                     "Area Covered by Thatched Roof"),
                    column.labels=c("Full", "Eligible",
                                    "Full", "Eligible",
                                    "Full", "Eligible"),
                    covariate.labels=c("Treatment Village", "High Saturation Sublocation", "Constant"))
cat(paste0(output, "\n"), file=paste0(OUT_DIR, "thatched_te.tex"))


% Table created by stargazer v.5.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Thu, Dec 05, 2019 - 02:28:34 AM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{6}{c}{\textit{Dependent variable:}} \\ 
\cline{2-7} 
\\[-1.8ex] & \multicolumn{2}{c}{No. of Houses w/ Thatched Roof} & \multicolumn{2}{c}{Mean Size of Houses w/ Thatched Roof} & \multicolumn{2}{c}{Area Covered by Thatched Roof} \\ 
 & Full & Eligible & Full & Eligible & Full & Eligible \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6)\\ 
\hline \\[-1.8ex] 
 Treatment Village & $-$0.258$^{***}$ & $-$0.263$^{***}$ & 3.785 & 3.560 & $-$31.966$^{***}$ & $-$32.532$^{***}$ \\ 
  & (0.064) & (0.065) & (3.158) & (3.538) & (8.961) & (9.386) \\ 
  & & & & & & \\ 
 High Saturation Sublocation & $-$0.134 & $-$0.131 & 3.696 & 3.471 & $-$17.933 & $-$17.404 \\ 
  & (0.101) & (0.105) & (3.076

In [10]:
reg1 <- felm(colored_sat_house ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg2 <- felm(colored_sat_house ~ treat + hi_sat | 0 | 0 | satcluster, df[df$p1_6_nonthatchedroof_BL == 0,])
reg3 <- felm(colored_sat_size_mean ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg4 <- felm(colored_sat_size_mean ~ treat + hi_sat | 0 | 0 | satcluster, df[df$p1_6_nonthatchedroof_BL == 0,])
reg5 <- felm(colored_sat_size_sum ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg6 <- felm(colored_sat_size_sum ~ treat + hi_sat | 0 | 0 | satcluster, df[df$p1_6_nonthatchedroof_BL == 0,])

output <- stargazer(reg1, reg2, reg3, reg4, reg5, reg6,
                    dep.var.labels=c("No. of Houses w/ Colored Roof",
                                     "Mean Size of Houses w/ Colored Roof",
                                     "Area Covered by Colored Roof"),
                    column.labels=c("Full", "Eligible",
                                    "Full", "Eligible",
                                    "Full", "Eligible"),
                    covariate.labels=c("Treatment Village", "High Saturation Sublocation", "Constant"))
cat(paste0(output, "\n"), file=paste0(OUT_DIR, "colored_te.tex"))


% Table created by stargazer v.5.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Thu, Dec 05, 2019 - 02:28:35 AM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{6}{c}{\textit{Dependent variable:}} \\ 
\cline{2-7} 
\\[-1.8ex] & \multicolumn{2}{c}{No. of Houses w/ Colored Roof} & \multicolumn{2}{c}{Mean Size of Houses w/ Colored Roof} & \multicolumn{2}{c}{Area Covered by Colored Roof} \\ 
 & Full & Eligible & Full & Eligible & Full & Eligible \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6)\\ 
\hline \\[-1.8ex] 
 Treatment Village & 0.002 & 0.003 & $-$28.251 & $-$5.505 & $-$3.794 & $-$0.569 \\ 
  & (0.010) & (0.010) & (32.219) & (37.528) & (5.763) & (5.735) \\ 
  & & & & & & \\ 
 High Saturation Sublocation & 0.035$^{*}$ & 0.032 & $-$53.915$^{*}$ & $-$74.188$^{**}$ & 15.965 & 12.351 \\ 
  & (0.020) & (0.020) & (29.101) & (32.168) & (9.